In [42]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id="stapesai/ssi-speech-emotion-recognition", repo_type = 'dataset',
                  allow_patterns="data/*.parquet", local_dir = 'ssi-speech-emotion-recognition')

In [40]:
from glob import glob
import polars as pl
import soundfile as sf

files = glob('ssi-speech-emotion-recognition/data/*.parquet')
len(files)

5

In [54]:
intensity = {
    'HI': 'high',
    'LO': 'low',
    'MD': 'medium',
    'UNK': 'unknown',
}

emotion = {
    'ANG': 'anger',
    'CAL': 'calm',
    'DIS': 'disgust',
    'FEA': 'fear',
    'HAP': 'happy',
    'NEU': 'neutral',
    'SAD': 'sadness',
    'SUR': 'surprise',
}

In [33]:
columns = ['text', 'emotion', 'emotion_intensity']

In [41]:
!mkdir ssi-speech-emotion-recognition-audio

In [55]:
import os

data = []
for f in files:
    f_ = f.replace('/', '_').replace('.parquet', '')
    df = pl.read_parquet(f)
    for i in range(len(df)):
        if df[i]['text'][0] == 'UNK':
            continue
        intens = df[i]['emotion_intensity'][0]
        if intens not in intensity:
            continue
        filename = os.path.join('ssi-speech-emotion-recognition-audio', f'{f_}_{i}.wav')
        
        with open(filename, 'wb') as fopen:
            fopen.write(df[i]['file_path'][0]['bytes'])
        
        data.append({
            'filename': filename,
            'text': df[i]['text'][0],
            'emotion': emotion[df[i]['emotion'][0]],
            'intensity': intensity.get(intens, intens),
        })
        
len(data)

11681

In [58]:
data[-1]

{'filename': 'ssi-speech-emotion-recognition-audio/ssi-speech-emotion-recognition_data_train-00001-of-00003_3331.wav',
 'text': 'Kids are talking by the door',
 'emotion': 'surprise',
 'intensity': 'medium'}

In [59]:
import json

with open('ssi-speech-emotion-recognition.json', 'w') as fopen:
    json.dump(data, fopen)